In [22]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer


In [23]:
df = pd.read_csv('PPR-cleaned-v2.csv')
df.head()

,Date,Town,County,Price,Not Full Market Price,VAT Exclusive,Description of Property,Province
0,2010-01-01,churchtown,Dublin,343000.0,No,No,Second-Hand,Leinster
1,2010-03-01,portlaoise,Laois,185000.0,No,Yes,New,Leinster
2,2010-04-01,dublin 14,Dublin,438500.0,No,No,Second-Hand,Leinster
3,2010-04-01,mornington,Meath,400000.0,No,No,Second-Hand,Leinster
4,2010-04-01,ranelagh,Dublin,425000.0,No,No,Second-Hand,Leinster


In [24]:
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421313 entries, 0 to 421312
Data columns (total 8 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   Date                     421313 non-null  datetime64[ns]
 1   Town                     421313 non-null  object        
 2   County                   421313 non-null  object        
 3   Price                    421313 non-null  float64       
 4   Not Full Market Price    421313 non-null  object        
 5   VAT Exclusive            421313 non-null  object        
 6   Description of Property  421313 non-null  object        
 7   Province                 421313 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 25.7+ MB


# Testing spaCy

In [34]:
# !pip install spacy
# !python -m spacy download en_core_web_sm

In [35]:
# import spacy

# nlp = spacy.load("en_core_web_sm")


In [36]:
# df = pd.read_csv('PPR-2010-2023.csv')
# df = df.drop('Property Size Description', axis=1)
# df = df.drop('Eircode',axis=1)

# def clean_price(string):
#     string = string.replace(",","")
#     return float(string[1:])
# Price = df['Price ()'].apply(clean_price)
# df = df.drop('Price ()',axis=1)
# df['Price'] = Price
# df['Date'] = df['Date of Sale (dd/mm/yyyy)']
# df = df.drop('Date of Sale (dd/mm/yyyy)',axis=1)

# replacement_dict = {
#     'Second-Hand Dwelling house /Apartment':'Second-Hand',
#     'New Dwelling house /Apartment':'New',
#     'Teach/Árasán Cónaithe Atháimhe':'Second-Hand',
#     'Teach/Árasán Cónaithe Nua':'New',
#     'Teach/?ras?n C?naithe Nua':'New'
# }
# df['Description of Property'] = df['Description of Property'].replace(replacement_dict)
# df = df[['Date','Address','County','Price','Not Full Market Price','VAT Exclusive','Description of Property']]
# df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)

In [37]:
# df['Town'] = ''

In [38]:
# for i, row in df.iterrows():
#     address = row['Address']
#     doc = nlp(address)
#     for ent in doc.ents:
#         if ent.label_ == "GPE":
#             df.at[i, 'Town'] = ent.text
#             break

# Encoding

In [32]:
# Encode the County column using one-hot encoding
county_ohe = OneHotEncoder(sparse=False)
county_encoded = county_ohe.fit_transform(df[['County']])

# Encode the Town column using a combination of label encoding and target encoding
town_le = LabelEncoder()
df['Town_le'] = town_le.fit_transform(df['Town'])
town_target_encoded = df.groupby('Town_le')['Price'].mean()
df['Town_target_encoded'] = df['Town_le'].replace(town_target_encoded)

# Encode the Province column using label encoding
province_le = LabelEncoder()
df['Province_encoded'] = province_le.fit_transform(df['Province'])

# Encode the Not Full Market Price column using binary encoding
nfmpe_lb = LabelBinarizer()
nfmpe_encoded = nfmpe_lb.fit_transform(df['Not Full Market Price'])

# Encode the Vat Exclusive column using binary encoding
ve_lb = LabelBinarizer()
ve_encoded = ve_lb.fit_transform(df['VAT Exclusive'])

# Encode the Description column using binary encoding
desc_lb = LabelBinarizer()
desc_encoded = desc_lb.fit_transform(df['Description of Property'])

# Concatenate the encoded columns to the original dataframe
encoded_df = pd.concat([df[['Price']], pd.DataFrame(county_encoded), 
                        df[['Town_target_encoded']], df[['Province_encoded']],
                        pd.DataFrame(nfmpe_encoded), pd.DataFrame(ve_encoded),
                        pd.DataFrame(desc_encoded)], axis=1)

/Users/pratik/opt/miniconda3/envs/SETU/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [33]:
encoded_df

,Price,0,1,2,3,4,5,6,7,8,...,21,22,23,24,25,Town_target_encoded,Province_encoded,0,0,0
0,343000.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,420663.500197,1,0,0,1
1,185000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,173212.860267,1,0,1,0
2,438500.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,598500.769416,1,0,0,1
3,400000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,260974.172342,1,0,0,1
4,425000.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,630923.056364,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421308,110000.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,135058.797065,0,0,0,1
421309,76000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,225056.345915,2,1,0,1
421310,40000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,161477.413789,3,0,0,1
421311,135000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,79772.236090,0,0,0,1
